# Install packages

In [ ]:
reload(basic_func)
reload(plot_trials)
null_arc_info_for_plotting_temp = null_arc_info_for_plotting.copy()
null_arc_info_for_plotting_temp['show_connect_path_ff'] = True
category_item.plot_trajectories(trials=category_item.sort_2_trials[current_i:current_i+5], null_arc_info_for_plotting=null_arc_info_for_plotting_temp)
current_i = current_i+5

In [ ]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2

# Import packages

In [ ]:
# if using google drive
%cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
%cd /Users/dusiyi/Documents/Multifirefly-Project

In [2]:
from importlib import reload
# reload(plot_trials)
# reload(intended_targets_classes)

In [ ]:
%load_ext autoreload
%autoreload 2


import sys
 sys.path.append('/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods')
from data_wrangling import basic_func, process_raw_data, base_processing_class, monkey_data_classes
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis import decision_making_class, decision_making_utils, cluster_replacement_utils, intended_targets_classes, cluster_replacement_class, process_GUAT_trials_class, GUAT_helper_class
from visualization import animation_func, animation_utils, plot_trials, plot_polar, additional_plots, plot_behaviors_utils, plot_statistics
from machine_learning import machine_learning_utils, hyperparameter_tuning_class
from machine_learning.RL.env_related import env_for_lstm, env_utils, base_env, collect_agent_data_utils
from machine_learning.RL.lstm import GRU_functions, LSTM_functions
from machine_learning.RL.SB3 import interpret_neural_network, sb3_for_multiff_class, rl_for_multiff_utils, SB3_functions
from non_behavioral_analysis import neural_analysis, eye_positions
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials


import os
import torch
import numpy as np
import matplotlib
from matplotlib import rc, cm
import matplotlib.pyplot as plt
import torch.nn as nn
from os.path import exists
import pandas as pd
from IPython.display import HTML
import matplotlib.ticker as ticker
from matplotlib import animation
from functools import partial
from math import pi

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

# Basic data

## Plotting kwargs

In [ ]:
trial_total_num = 2
PLAYER = "monkey"

classic_plot_kwargs = {'player': PLAYER,
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'show_eye_positions': True,
                       'show_eye_positions_on_the_right': True,
                       'show_connect_path_eye_positions': True,
                       #=== below is different from animation_plot_kwargs
                       'hitting_arena_edge_ok': False,
                       'trial_too_short_ok': False}

combined_plot_kwargs = {'player': PLAYER,
                        'combined_plot': True,
                        'show_alive_fireflies': False,
                        'show_title': False,
                        'show_start': False}

plot_chunks_kwargs = {'player': PLAYER,
                      'show_stops': True,
                      'show_believed_target_positions': True,
                      'show_reward_boundary': True,
                      'show_scale_bar': True,
                      'hitting_arena_edge_ok': True,
                      'trial_too_short_ok': True,
                      'show_connect_path_ff': True,
                      'trail_color_var': 'abs_ddw',
                      'show_colorbar': True}


plot_polar_kwargs = {'rmax': 400,
                    'show_visible_ff': True,
                    'hitting_arena_edge_ok': True,
                    'return_axes': True,
                    'colors_show_overall_time': True,
                    'show_ff_in_memory': True,
                    'show_target_at_being_caught': True
}

print("player is", PLAYER)

animation_plot_kwargs = classic_plot_kwargs.copy()
animation_plot_kwargs['as_part_of_animation'] = True
animation_plot_kwargs['show_eye_positions_on_the_right'] = False   
animation_plot_kwargs['hitting_arena_edge_ok'] = True
animation_plot_kwargs['trial_too_short_ok'] = True
animation_plot_kwargs['images_dir'] = None

## Retrieve monkey data

In [ ]:
PLAYER = "monkey"
monkey_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330"
data_item = monkey_data_classes.ProcessMonkeyData(raw_data_folder_path=monkey_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()
data_item.make_PlotTrials_args()
data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_sorted = data_item.ff_caught_T_sorted
caught_ff_num = len(ff_caught_T_sorted)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index

monkey_name = data_item.model_folder_name.split('/')[2]
data_name = data_item.model_folder_name.split('/')[3]



# data_item.make_or_retrieve_all_trial_patterns(exists_ok=True)
# data_item.make_or_retrieve_pattern_frequencies(exists_ok=True)
# data_item.make_or_retrieve_all_trial_features(exists_ok=True)
# data_item.make_or_retrieve_feature_statistics(exists_ok=True)
# data_item.make_info_of_monkey()
# data_item.make_or_retrieve_target_closest()
# data_item.make_or_retrieve_target_angle_smallest()
# data_item.make_distance_dataframe()
# data_item.make_angle_dataframe()

# target_closest = data_item.target_closest
# target_angle_smallest = data_item.target_angle_smallest
# all_trial_patterns = data_item.all_trial_patterns
# all_trial_features = data_item.all_trial_features
# pattern_frequencies = data_item.pattern_frequencies
# feature_statistics = data_item.feature_statistics
# distance_dataframe = data_item.distance_dataframe
# angle_dataframe = data_item.angle_dataframe
# trial_vs_distance = data_item.trial_vs_distance
# trial_vs_angle = data_item.trial_vs_angle
# info_of_monkey = data_item.info_of_monkey
# max_point_index = data_item.max_point_index
# min_point_index = data_item.min_point_index

# data_item.make_PlotTrials_args()
# data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)



## Plotting args

In [17]:
PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_sorted)


plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_sorted,
                    ff_flash_sorted,)

In [ ]:
# chunk_numbers = range(20, 30)
# additional_kwargs = {'show_connect_path_ff_memory': True,
#                      'show_connect_path_ff_except_targets': True,
#                      'show_path_when_target_visible': True,
#                      'show_connect_path_eye_positions': False}
# data_item.make_animation_of_chunks(points_w_more_than_2_ff, monkey_information, chunk_numbers = chunk_numbers, sampling_frame_ratio = 3, additional_kwargs=additional_kwargs, exists_ok=True)

# Retrieve data

## Basic parameters

In [ ]:
NEW_DATASET = False
MONKEY_DATA = True
NO_PLOT_NEEDED = True
raw_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330"
data_folder_name = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330/processed_data"



#data_num = 19


# # for agent
# PLAYER = "agent"
# NEW_DATASET = True
# MONKEY_DATA = False
# NO_PLOT_NEEDED = True
# data_folder_name = "RL/LSTM_July_29"
# data_num = 721
# trial_total_num = 2

## Which players to use?

In [ ]:
use_LSTM = False
use_SB3 = False
use_monkey = True

## SB3 agent data

In [ ]:
model_folder_name = "RL/SB3_stored_models/all_agents/Aug_10_3"
retrieve_dir = "RL/SB3_stored_models/all_agents/Aug_10_3"

In [ ]:
!pip install wheel==0.38.4
!pip3 install setuptools==65.5.0
!pip install stable_baselines3
from stable_baselines3 import SAC

env_kwargs = {"action_noise_std": 0.2, 
              "ffxy_noise_std": 4, 
              "num_obs_ff": 2,
              "full_memory": 3}

if use_SB3:

    PLAYER = "agent"

    model_folder_name = "RL/SB3_stored_models/all_agents/A0.2_O4_ff2_memory3/"
    data_folder_name = model_folder_name
    os.makedirs(model_folder_name, exist_ok=True)


    env = env.CollectInformation(**env_kwargs)
    env.reset()

    # For direct training
    sac_model = SAC("MlpPolicy", 
                env,
                gamma=0.995,
                learning_rate=0.0015,
                batch_size=1024,
                target_update_interval=50,
                buffer_size=1000000,
                learning_starts=10000,
                train_freq=10,
                ent_coef='auto',
                policy_kwargs=dict(activation_fn=nn.Tanh, net_arch=[128, 128])
                    )


    path = os.path.join(model_folder_name, 'best_model.zip')
    sac_model = sac_model.load(path, env=env)
    
    monkey_information, ff_flash_sorted, ff_caught_T_sorted, ff_believed_position_sorted, ff_real_position_sorted, \
        ff_life_sorted, ff_flash_end_sorted, caught_ff_num, total_ff_num, obs_ff_indices_in_ff_dataframe, sorted_indices_all, ff_in_obs_df \
        =collect_agent_data_utils.collect_agent_data_func(env, sac_model, n_steps = 15000)

    ff_dataframe_args = (monkey_information, ff_caught_T_sorted, ff_flash_sorted,  ff_real_position_sorted, ff_life_sorted)
    ff_dataframe_kargs = {"max_distance": 400, "data_folder_name": data_folder_name, "num_missed_index": 0}
    ff_dataframe = make_ff_dataframe.make_ff_dataframe_func(*ff_dataframe_args, **ff_dataframe_kargs, player = "agent", 
                                     obs_ff_indices_in_ff_dataframe = obs_ff_indices_in_ff_dataframe, ff_in_obs_df = ff_in_obs_df)
    min_point_index, max_point_index = np.min(np.array(ff_dataframe['point_index'])), np.max(np.array(ff_dataframe['point_index']))

    filepath = data_folder_name + '/patterns_and_features/all_trial_features.csv'
    if (PLAYER == "monkey") & (exists(filepath)):
        all_trial_features = pd.read_csv(filepath)
    else:
        all_trial_features = organize_patterns_and_features.make_all_trial_features(ff_dataframe, monkey_information, ff_caught_T_sorted, cluster_around_target_indices, ff_real_position_sorted, ff_believed_position_sorted,  data_folder_name = data_folder_name)



## LSTM agent data

In [ ]:
if use_LSTM:
      
    from LSTM_agent_params import *
    PLAYER = "agent"

    model_folder_name = "RL_models/LSTM_stored_models/all_agents/LSTM_Aug_1_24"
    data_folder_name = model_folder_name
    os.makedirs(model_folder_name, exist_ok=True)


    replay_buffer_size=100
    replay_buffer = LSTM_functions.ReplayBufferLSTM2(replay_buffer_size)
    sac_model=LSTM_functions.SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                          gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                          batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                          target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq, device = device)


    # Load agent
    sac_model.load_model(model_folder_name)

    env = env.CollectInformationLSTM()
    env.flash_on_interval = 0.3
    env.distance2center_cost = 0
    sac_model.load_model(model_folder_name)



    monkey_information, ff_flash_sorted, ff_caught_T_sorted, ff_believed_position_sorted, \
              ff_real_position_sorted, ff_life_sorted, ff_flash_end_sorted, caught_ff_num, total_ff_num, \
              obs_ff_indices_in_ff_dataframe, sorted_indices_all \
              = collect_agent_data_utils.collect_agent_data_func(env, sac_model, n_steps = 5000, LSTM = True, hidden_dim= hidden_dim, device = device, deterministic = False)

    ff_dataframe_args = (monkey_information, ff_caught_T_sorted, ff_flash_sorted,  ff_real_position_sorted, ff_life_sorted)
    ff_dataframe_kargs = {"max_distance": 400, "data_folder_name": data_folder_name, "num_missed_index": 0}
    ff_dataframe = make_ff_dataframe.make_ff_dataframe_func(*ff_dataframe_args, **ff_dataframe_kargs, player = "agent",  \
                                      obs_ff_indices_in_ff_dataframe = obs_ff_indices_in_ff_dataframe, ff_in_obs_df = ff_in_obs_df)
    min_point_index, max_point_index = np.min(np.array(ff_dataframe['point_index'])), np.max(np.array(ff_dataframe['point_index']))

    filepath = data_folder_name + '/patterns_and_features/all_trial_features.csv'
    if (PLAYER == "monkey") & (exists(filepath)):
        all_trial_features = pd.read_csv(filepath)
    else:
        all_trial_features = organize_patterns_and_features.make_all_trial_features(ff_dataframe, monkey_information, ff_caught_T_sorted, cluster_around_target_indices, ff_real_position_sorted, ff_believed_position_sorted,  data_folder_name = data_folder_name)



## Monkey data

In [ ]:
if use_monkey:

    PLAYER = "monkey"
    raw_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0330"
    data_item = monkey_data_classes.ProcessMonkeyData(raw_data_folder_path=raw_data_folder_path)
    data_item.retrieve_or_make_monkey_data(exists_ok=True)
    data_item.make_or_retrieve_ff_dataframe(num_missed_index=0, exists_ok=True)
    monkey_information = data_item.monkey_information
    ff_dataframe = data_item.ff_dataframe
    min_point_index, max_point_index = np.min(np.array(ff_dataframe['point_index'])), np.max(np.array(ff_dataframe['point_index']))

    caught_ff_num = len(ff_caught_T_sorted)
    cluster_around_target_trials, cluster_around_target_indices, cluster_around_target_positions = pattern_by_trials.cluster_around_target_func(ff_dataframe, caught_ff_num, ff_caught_T_sorted, ff_real_position_sorted, max_time_apart = 1.25)


    info_of_monkey = {
            "monkey_information": monkey_information,
            "ff_dataframe": ff_dataframe,
            "ff_caught_T_sorted": ff_caught_T_sorted,
            "ff_real_position_sorted": ff_real_position_sorted,
            "ff_believed_position_sorted": ff_believed_position_sorted,
            "ff_life_sorted": ff_life_sorted,
            "ff_flash_sorted": ff_flash_sorted,
            "ff_flash_end_sorted": ff_flash_end_sorted,
            "cluster_around_target_indices": cluster_around_target_indices
    }


    filepath = data_folder_name + '/patterns_and_features/all_trial_features.csv'
    if (PLAYER == "monkey") & (exists(filepath)):
        all_trial_features = pd.read_csv(filepath)
    else:
        all_trial_features = organize_patterns_and_features.make_all_trial_features(ff_dataframe, monkey_information, ff_caught_T_sorted, cluster_around_target_indices, ff_real_position_sorted, ff_believed_position_sorted)

## Find patterns

In [12]:
n_ff_in_a_row = pattern_by_trials.n_ff_in_a_row_func(ff_believed_position_sorted, distance_between_ff = 50)
two_in_a_row = np.where(n_ff_in_a_row==2)[0]
two_in_a_row_simul, two_in_a_row_non_simul = pattern_by_trials.whether_current_and_last_targets_are_captured_simultaneously(two_in_a_row, ff_caught_T_sorted)

on_before_last_one_trials = pattern_by_trials.on_before_last_one_func(ff_flash_end_sorted, ff_caught_T_sorted, caught_ff_num)
on_before_last_one_simul, on_before_last_one_non_simul = pattern_by_trials.whether_current_and_last_targets_are_captured_simultaneously(on_before_last_one_trials, ff_caught_T_sorted)

visible_before_last_one_trials = pattern_by_trials.visible_before_last_one_func(ff_dataframe)

disappear_latest_trials = pattern_by_trials.disappear_latest_func(ff_dataframe)

# cluster_exist_trials, cluster_dataframe_point, cluster_dataframe_trial = cluster_analysis.clusters_of_ffs_func(point_vs_cluster, monkey_information, ff_caught_T_sorted)

# waste_cluster_trials = np.intersect1d(cluster_exist_trials+1, np.where(n_ff_in_a_row == 1)[0])

cluster_around_target_trials, cluster_around_target_indices, cluster_around_target_positions = pattern_by_trials.cluster_around_target_func(ff_dataframe, caught_ff_num, ff_caught_T_sorted, ff_real_position_sorted, max_time_apart = 1.25)

waste_cluster_around_target_trials = np.intersect1d(cluster_around_target_trials+1, np.where(n_ff_in_a_row == 1)[0])

ignore_sudden_flash_trials, ignore_sudden_flash_indices, ignore_sudden_flash_indices_for_anim, ignored_ff_target_pairs = pattern_by_trials.ignore_sudden_flash_func(ff_dataframe, ff_real_position_sorted, max_point_index, max_ff_distance_from_monkey = 50)
ignore_sudden_flash_time = monkey_information['monkey_t'][ignore_sudden_flash_indices]

try_a_few_times_trials, TAFT_indices_df, TAFT_trials_df, try_a_few_times_indices_for_anim = find_GUAT_or_TAFT_trials.try_a_few_times_func(monkey_information, ff_caught_T_sorted, ff_real_position_sorted, max_point_index=max_point_index)

give_up_after_trying_trials, GUAT_indices_df, GUAT_trials_df, GUAT_point_indices_for_anim = find_GUAT_or_TAFT_trials.give_up_after_trying_func\
    (monkey_information, ff_caught_T_sorted, ff_real_position_sorted, max_point_index=max_point_index)


# Classic plots

In VScode, for animation to work successfully, HTML(data_instance.anim.to_html5_video())
has to be the last line of the code, and it can’t be within a for loop or if/else statement


## Visible before last one

In [14]:
trial_total_num = 2

In [ ]:

num_trials = 1
category = visible_before_last_one_trials
category = category[category > num_trials]
category_name = 'visible_before_last_one'




if len(category) > 0:
    for currentTrial in category[:trial_total_num]:
        with basic_func.initiate_plot(7, 7, 100):
            duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]

            plot_trials.PlotTrials(duration, 
                        *PlotTrials_args,
                        **classic_plot_kwargs,
                        currentTrial = currentTrial,
                        num_trials = num_trials,                                  
                        show_connect_path_ff_except_targets = True,
                        show_path_when_target_visible = True,
                        )

            plt.show()  
            


temp_animation_plot_kwargs = animation_plot_kwargs.copy()
#temp_animation_plot_kwargs['show_connect_path_ff_except_targets'] = True
temp_animation_plot_kwargs['show_path_when_target_visible'] = True
data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None, dt=0.016)
HTML(data_item.anim.to_html5_video())




## Disappear latest

In [ ]:
num_trials = 1
category = disappear_latest_trials
category = category[category > num_trials]
category_name = 'disappear_latest'


if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]

        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,
                   show_connect_path_ff = True,
        )

        # temp_animation_plot_kwargs = animation_plot_kwargs.copy()
        # data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        

        plt.show()  
        

## 2 in a row

In [ ]:
num_trials = 2 
category = two_in_a_row
category = category[category > num_trials]
#category = two_in_a_row_simuly
#category = two_in_a_row_non_simul
#category = used_cluster  --> two_in_a_row_non_simul ^ on_before_last_one
category_name = 'two_in_a_row'



if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num]:
    #for currentTrial in [17, 45]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,                   
                   show_connect_path_ff_except_targets = True,
                   show_path_when_target_visible = True,
                   )
        
        temp_animation_plot_kwargs = animation_plot_kwargs.copy()   
        #temp_animation_plot_kwargs['show_connect_path_ff_except_targets'] = True
        temp_animation_plot_kwargs['show_path_when_target_visible'] = True     
        data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        #HTML(.to_html5_video())
        plt.show()  
        
        data_item.anim

## Waste cluster around last target

In [ ]:
num_trials = 2
category = waste_cluster_around_target_trials
category = category[category > num_trials]
category_name = 'waste_cluster_around_target'



if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        
        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,
                   show_connect_path_ff = True,
                   trial_to_show_cluster_around_target = "previous",
                   )

        temp_animation_plot_kwargs = animation_plot_kwargs.copy()
        temp_animation_plot_kwargs['trial_to_show_cluster_around_target'] = "previous"
        data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        

        plt.show()  
        

## Try a few times

In [ ]:
category = try_a_few_times_trials
category = category[category > num_trials]
category_name = 'try_a_few_times'
num_trials = 2


if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num+5]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,
                   show_connect_path_ff = True,                
                   )

        plt.show()  
        
        
        temp_animation_plot_kwargs = animation_plot_kwargs.copy()
        data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        


## Give up after trying

In [ ]:
category = give_up_after_trying_trials
category = category[category > num_trials]
category_name = 'give_up_after_trying'


num_trials = 2


if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,
                   show_connect_path_ff = True,
                   )
        

        plt.show()  
        

        temp_animation_plot_kwargs = animation_plot_kwargs.copy()
        data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        

## Ignore sudden flash

In [ ]:
category = ignore_sudden_flash_trials
category = category[category > num_trials]
category_name = 'ignore_sudden_flash'

num_trials = 1

ignored_ff_target_pairs_index_only = ignored_ff_target_pairs[['ff_index']]

if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
    for currentTrial in category[:trial_total_num]:
        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]

        ignored_ff_indices = np.array(ignored_ff_target_pairs_index_only.loc[currentTrial]).reshape(-1).tolist()

        plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **classic_plot_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,                   
                   show_connect_path_ff = True,
                   indices_of_ff_to_mark = ignored_ff_indices,             
                   )
        

        plt.show()  
        


        temp_animation_plot_kwargs = animation_plot_kwargs.copy()
        temp_animation_plot_kwargs['indices_of_ff_to_mark'] = ignored_ff_indices
        data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        

### Relative locations of all the ignored ff

In [ ]:
import math
rmax = 55
print("Using ff_angle")

fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
ax.set_theta_zero_location("N")
ax.set_rlabel_position(292.5)
ax.set_ylim(0, rmax)
# Draw the boundary of the monkey's vision (use width = np.pi*4/9 for 40 degrees of vision)
ax.bar(0, rmax, width=np.pi/2, bottom=0.0, color="grey", alpha=0.1)
plt.setp(ax, rorigin=0, rmin=0, rmax=rmax)   # rmax can be changed

# Change the labels for the angles 
labels = list(ax.get_xticks())
labels[5], labels[6], labels[7] = -labels[3], -labels[2], -labels[1]
labels_degrees = [str(int(math.degrees(label))) + chr(176) for label in labels]
ax.set_xticks(ax.get_xticks().tolist())
ax.set_xticklabels(labels_degrees)

# Plot the locations of the FFs
ax.scatter(ignored_ff_target_pairs['ff_angle'], ignored_ff_target_pairs['ff_distance'], marker='.', s=20, alpha=1)
plt.show()

In [ ]:
rmax = 55
print("Using ff_angle_boundary")

fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
ax.set_theta_zero_location("N")
ax.set_rlabel_position(292.5)
ax.set_ylim(0, rmax)
# Draw the boundary of the monkey's vision (use width = np.pi*4/9 for 40 degrees of vision)
ax.bar(0, rmax, width=np.pi/2, bottom=0.0, color="grey", alpha=0.1)
plt.setp(ax, rorigin=0, rmin=0, rmax=rmax)   # rmax can be changed

# Change the labels for the angles 
labels = list(ax.get_xticks())
labels[5], labels[6], labels[7] = -labels[3], -labels[2], -labels[1]
labels_degrees = [str(int(math.degrees(label))) + chr(176) for label in labels]
ax.set_xticks(ax.get_xticks().tolist())
ax.set_xticklabels(labels_degrees)

# Plot the locations of the FFs
ax.scatter(ignored_ff_target_pairs['ff_angle_boundary'], ignored_ff_target_pairs['ff_distance'], marker='.', s=20, alpha=1)

plt.show()

# Subplots

## Visible before last one

In [ ]:
trial_total_num = 10
PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_sorted)

category = visible_before_last_one_trials
subplots_kwargs = classic_plot_kwargs.copy()
subplots_kwargs['show_eye_positions'] = False
subplots_kwargs['show_eye_positions_on_the_right'] = False

num_trials = 2
k = 1
returned_info['whether_plotted'] = True
if len(category) > 0:
  with basic_func.initiate_plot(10, 10, 100):
      fig = plt.figure()
      for currentTrial in category[:trial_total_num]:
        if returned_info['whether_plotted'] is True:
            axes = fig.add_subplot(2,2,k)

        duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        
        returned_info = plot_trials.PlotTrials(duration, 
                   *PlotTrials_args,
                   **subplots_kwargs,
                   currentTrial = currentTrial,
                   num_trials = num_trials,
                   fig = fig,
                   axes = axes,
                   subplots = True,
                   show_connect_path_ff_except_targets = True,
                   show_path_when_target_visible = True,
                   )
        if returned_info['returned_info['whether_plotted']'] is True:
            k += 1
        if k == 5:
            plt.show()  
            
            break
        

        

# Combined plots

## Visible before last one

In [ ]:
category = visible_before_last_one_trials
num_trials = 2
xmin, xmax, ymin, ymax = 0, 0, 0, 0             
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)      
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_path_when_target_visible = True,
                                                          )
          axes = returned_info['axes']
          if returned_info['returned_info['whether_plotted']'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
              axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, s=40, alpha=0.85, zorder=5)
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Disappear latest

In [ ]:
category = disappear_latest_trials
num_trials = 1
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(10, 10, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      print("axes", axes)
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,   
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_path_when_target_visible = True,
                                                          show_believed_target_positions = True,
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      


## 2 in a row

In [ ]:
category = two_in_a_row
num_trials = 2
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
              axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, s=40, alpha=0.85, zorder=5)
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Cluster around targets

In [ ]:
import random
category = cluster_around_target_trials
num_trials = 1
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,     
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_trajectory = False
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              cluster_around_target_rotated = np.matmul(R, np.stack((cluster_around_target_positions[currentTrial].T[0], cluster_around_target_positions[currentTrial].T[1])))
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial].T[0], ff_believed_position_sorted[currentTrial].T[1])))
              current_color = '#%06X' % random.randint(0, 0xFFFFFF)
              axes.scatter(cum_mxy_rotated[0]-x0, cum_mxy_rotated[1]-y0,marker = 'o',s = 2, alpha= 0.7, color = current_color)
              axes.scatter(cluster_around_target_rotated[0]-x0, cluster_around_target_rotated[1]-y0, marker='o', s=30, color = current_color) 
              axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, marker='o', s=30, color = current_color) 
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Waste cluster around last target

In [ ]:
category = waste_cluster_around_target_trials
num_trials = 2
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,
                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_trajectory = False
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              cluster_around_target_rotated = np.matmul(R, np.stack((cluster_around_target_positions[currentTrial-1].T[0], cluster_around_target_positions[currentTrial-1].T[1])))
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-1].T[0], ff_believed_position_sorted[currentTrial-1].T[1])))
              current_color = '#%06X' % random.randint(0, 0xFFFFFF)
              axes.scatter(cum_mxy_rotated[0]-x0, cum_mxy_rotated[1]-y0,marker = 'o',s = 2, alpha= 0.7, color = current_color)
              axes.scatter(cluster_around_target_rotated[0]-x0, cluster_around_target_rotated[1]-y0, marker='o', s=30, color = current_color) 
              axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, marker='o', s=30, color = current_color) 
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Try a few times

In [ ]:
category = try_a_few_times_trials
num_trials = 1
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[20:30]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,
                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_believed_target_positions = True,
                                                          show_path_when_target_visible = True,
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              current_color = '#%06X' % random.randint(0, 0xFFFFFF)
              # Plot believed target positions
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
              axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, s=45, alpha=0.75, color=current_color, zorder=3) 
              # Plot stops
              cum_iloc_indices = np.where((monkey_information['monkey_t'] > duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
              zerospeed_index = np.where(monkey_information['monkey_speeddummy'].iloc[cum_iloc_indices]==0)[0]
              axes.scatter(cum_mxy_rotated[0][zerospeed_index]-x0, cum_mxy_rotated[1][zerospeed_index]-y0,marker = '*',s = 80, color = current_color, alpha=0.8, zorder=3)
              # Update the limits 
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)


      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Give up after trying

In [ ]:
category = give_up_after_trying_trials
num_trials = 1
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[:50]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,
                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_path_when_target_visible = True,
                                                          show_believed_target_positions = True,
                                                          show_stops = True
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

## Ignore sudden flash

In [ ]:
category = ignore_sudden_flash_trials
num_trials = 1
xmin, xmax, ymin, ymax = 0, 0, 0, 0
               
if len(category) > 0:
  with basic_func.initiate_plot(7, 7, 100):
      fig = plt.figure()
      axes = fig.add_subplot(111)
      for currentTrial in category[:30]:
          duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
          ignored_ff_indices = np.array(ignored_ff_target_pairs_index_only.loc[currentTrial]).reshape(-1).tolist()
          returned_info = plot_trials.PlotTrials(duration, 
                                                          *PlotTrials_args,
                                                          
                                                          **combined_plot_kwargs,
                                                          currentTrial = currentTrial,
                                                          num_trials = num_trials,
                                                          fig = fig,
                                                          axes = axes,
                                                          show_path_when_target_visible = True,
                                                          show_connect_path_ff_except_targets = True,
                                                          connect_path_ff_max_distance = 50,
                                                          show_believed_target_positions = True,
                                                          indices_of_ff_to_mark = ignored_ff_indices,   
                                                          )
          axes = returned_info['axes']
          if returned_info['whether_plotted'] is True:
              x0, y0 = cum_mxy_rotated[0][0], cum_mxy_rotated[1][0]
              ff_believed_position_rotated = np.matmul(R, np.stack((ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[0], ff_believed_position_sorted[currentTrial-num_trials+1:currentTrial+1].T[1])))
              #axes.scatter(ff_believed_position_rotated[0]-x0, ff_believed_position_rotated[1]-y0, s=40, alpha=0.85, zorder=5)
              xmin, ymin, xmax, ymax = plot_behaviors_utils.update_plot_limits(xmin, ymin, xmax, ymax, cum_mxy_rotated)

      axes.set_xlim((xmin-75, xmax+75))
      axes.set_ylim((ymin-25, ymax+75))
      axes.xaxis.set_major_locator(ticker.NullLocator())
      plt.show()  
      

# Polar plots

In [ ]:
num_trials = 1
for currentTrial in range(100, 105):
    print(currentTrial)
    #duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
    duration = [ff_caught_T_sorted[currentTrial]-1.25, ff_caught_T_sorted[currentTrial]]


    plot_polar.PlotPolar(duration,
              monkey_information,
              ff_dataframe, 
              ff_life_sorted,
              ff_real_position_sorted,
              ff_caught_T_sorted,
              ff_flash_sorted,
              rmax = 100,
              currentTrial = currentTrial,
              num_trials = num_trials,
              show_visible_ff = True,
              show_visible_target = True,
              # show_ff_in_memory = True,
              # show_target_in_memory = True,
              ff_colormap = 'viridis',
              target_colormap = 'viridis',
              show_alive_ff = True,
              colors_show_overall_time = True,
              show_all_positions_of_all_fireflies = True,
              show_colorbar = True,
                )

# Animation

## Overhead view

### prepare for animation

In [74]:
PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_sorted)

animation_plot_kwargs = classic_plot_kwargs.copy()
animation_plot_kwargs['show_eye_positions_on_the_right'] = False              

In [ ]:
currentTrial = 109
num_trials = 1
duration = None
currentTrial, num_trials, duration = basic_func.find_currentTrial_or_num_trials_or_duration(ff_caught_T_sorted, currentTrial, num_trials, duration)

k = 3
fig, ax = plt.subplots()
num_frames, anim_monkey_info, flash_on_ff_dict, alive_ff_dict, believed_ff_dict, new_num_trials, ff_dataframe_anim\
            = animation_utils.prepare_for_animation(ff_dataframe, ff_caught_T_sorted, ff_life_sorted, ff_believed_position_sorted, 
                                    ff_real_position_sorted, ff_flash_sorted, monkey_information, k=k, currentTrial=currentTrial, num_trials=num_trials)
print("Number of frames is:", num_frames)

In [ ]:
fig = plt.figure(figsize=(14.5, 7))
animation_plot_kwargs['hitting_arena_edge_ok'] = True
animation_plot_kwargs['show_connect_path_ff'] = True
animation_plot_kwargs['trial_too_short_ok'] = True
axes = fig.add_subplot(1, 2, 1)
# axes.remove()
returned_info = plot_trials.PlotTrials(duration, 
                        fig=fig,
                        axes=axes,
                        *PlotTrials_args,
                        **animation_plot_kwargs,
                        currentTrial = currentTrial,
                        num_trials = num_trials,           
                        )
axes = returned_info['axes']
fig.add_axes(axes)
ax = fig.add_subplot(1, 2, 2)
fig.tight_layout()
num_frames, anim_monkey_info, flash_on_ff_dict, alive_ff_dict, believed_ff_dict, new_num_trials, ff_dataframe_anim\
            = animation_utils.prepare_for_animation(ff_dataframe, ff_caught_T_sorted, ff_life_sorted, ff_believed_position_sorted, 
                                    ff_real_position_sorted, ff_flash_sorted, monkey_information, k=k, currentTrial=currentTrial, num_trials=num_trials)
print("Number of frames is:", num_frames)

### make animation

In [ ]:
#plt.rcParams['animation.ffmpeg_path'] = '/Users/dusiyi/.conda/envs/Multifirefly-Project/bin/ffmpeg'
animate_func = partial(animation_func.animate, ax=ax, anim_monkey_info=anim_monkey_info, ff_dataframe_anim=ff_dataframe_anim, \
                       flash_on_ff_dict=flash_on_ff_dict, alive_ff_dict=alive_ff_dict, believed_ff_dict=believed_ff_dict, margin = 400,
                       plot_eye_position=True, set_xy_limits=True)
anim = animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim.to_html5_video())

In [ ]:

num_trials = 1
category = visible_before_last_one_trials
category = category[category > num_trials]
category_name = 'visible_before_last_one'




if len(category) > 0:
    for currentTrial in category[:trial_total_num]:
        with basic_func.initiate_plot(7, 7, 100):
            duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]

            plot_trials.PlotTrials(duration, 
                        *PlotTrials_args,
                        **classic_plot_kwargs,
                        currentTrial = currentTrial,
                        num_trials = num_trials,                                  
                        show_connect_path_ff_except_targets = True,
                        show_path_when_target_visible = True,
                        )

            plt.show()  
            


temp_animation_plot_kwargs = animation_plot_kwargs.copy()
#temp_animation_plot_kwargs['show_connect_path_ff_except_targets'] = True
temp_animation_plot_kwargs['show_path_when_target_visible'] = True
data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None, dt=0.016)
HTML(data_item.anim.to_html5_video())




### make animation with annotation

Make sure that prepare_for_animation has been run first

In [ ]:
reload(animation_func)

In [ ]:

annotation_info = animation_utils.make_annotation_info(caught_ff_num+1, max_point_index, n_ff_in_a_row, visible_before_last_one_trials, disappear_latest_trials, \
                                        ignore_sudden_flash_indices, give_up_after_trying_indices, try_a_few_times_indices)
animate_annotated_func = partial(animation_func.animate_annotated, ax=ax, anim_monkey_info=anim_monkey_info, margin=100, ff_dataframe_anim=ff_dataframe_anim, \
                                   flash_on_ff_dict=flash_on_ff_dict, alive_ff_dict=alive_ff_dict, believed_ff_dict=believed_ff_dict, ff_caught_T_sorted=ff_caught_T_sorted, annotation_info=annotation_info)
anim_annotated = animation.FuncAnimation(fig, animate_annotated_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim_annotated.to_html5_video())

## Polar

In [ ]:
currentTrial = 101
num_trials = 1
filename = f"Trials {currentTrial-num_trials+1}-{currentTrial}"
print(filename)
k = 3
rmax = 100
colors_Reds = plt.get_cmap("Reds")(np.linspace(0,1,101))
colors_YlGn = plt.get_cmap("YlGn")(np.linspace(0,1,101))
cum_iloc_indices = np.where((monkey_information['monkey_t'] > ff_caught_T_sorted[currentTrial-num_trials]) & 
                       (monkey_information['monkey_t'] <= ff_caught_T_sorted[currentTrial]))

if len(cum_iloc_indices) > 0:
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
    ax = plot_behaviors_utils.set_polar_background_for_animation(ax, rmax)

    ff_in_time_frame, ff_visible, ff_in_memory = animation_func.subset_ff_dataframe(ff_dataframe, currentTrial, num_trials)
    anim_indices = cum_iloc_indices[0][0:-1:k]
    num_frames = anim_indices.size
    animate_func = partial(animation_func.animate_polar, ax=ax, anim_indices=anim_indices, rmax=400, ff_in_time_frame=ff_in_time_frame, ff_visible=ff_visible, ff_in_memory=ff_in_memory)
    anim = animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=int(16.5*k), repeat=True) 

    #gif_dir = '/content/gdrive/My Drive/fireflies_anim/???'
    #anim.save(f"{gif_dir}/{filename}.gif", writer='pillow', fps=60)
else:
    print("Please try another number for currentTrial, or increase num_trials.")



In [ ]:
# For some reason, I need to run the following in a separate cell; I can't combine it with the previous cell.
print("Number of frames is", num_frames)
HTML(anim.to_html5_video())

# Other distributions

## Stop speeds

In [ ]:
all_min_speed = []
all_min_dw = []
all_corr_dw = []
for time in ff_caught_T_sorted:
  duration = [time-0.5, time+0.5]
  cum_iloc_indices = np.where((monkey_information['monkey_t'] >= duration[0]) & (monkey_information['monkey_t'] <= duration[1]))[0]
  cum_t, cum_angles = np.array(monkey_information['monkey_t'].iloc[cum_iloc_indices]), np.array(monkey_information['monkey_angles'].iloc[cum_iloc_indices])
  cum_mx, cum_my = np.array(monkey_information['monkey_x'].iloc[cum_iloc_indices]), np.array(monkey_information['monkey_y'].iloc[cum_iloc_indices])
  cum_speed, cum_speeddummy = np.array(monkey_information['monkey_speed'].iloc[cum_iloc_indices]), np.array(monkey_information['monkey_speeddummy'].iloc[cum_iloc_indices])
  cum_dw = np.array(monkey_information['monkey_dw'].iloc[cum_iloc_indices])
  min_speed = min(cum_speed)
  corresponding_dw = cum_dw[cum_speed==min_speed]
  all_min_speed.append(min_speed)
  all_min_dw.append(min(np.abs(cum_dw)))
  all_corr_dw.append(round(abs(corresponding_dw).item(), 4))

In [ ]:
import seaborn as sns
fig = plt.subplots()
sns.histplot(all_min_speed)
plt.title("Distribution of min speed around catching targets")
plt.show()



In [ ]:
fig = plt.subplots()
sns.histplot(all_min_dw)
plt.title("Distribution of min abs dw around catching targets")
plt.show()


In [ ]:
fig = plt.subplots()
sns.histplot(all_corr_dw, binwidth=0.0002)
plt.title("Distribution of abs dw corresponding to min speed around catching targets")
plt.xlim(0, 0.004)
plt.show()


## All ff in a session

In [ ]:
with basic_func.initiate_plot(7, 7, 100):
    plt.scatter(ff_real_position_sorted[:,0], ff_real_position_sorted[:,1], s=10)
    plt.show()
    

## ff distribution over time

In [ ]:
plot_behaviors_utils.plot_ff_distribution_in_arena(ff_real_position_sorted, ff_life_sorted, ff_caught_T_sorted, images_dir=None)

# Appendix

## Retrieve more data

In [ ]:
all_trial_features_lstm = pd.read_csv('RL_models/LSTM_stored_models/all_agents/LSTM_Aug_1_24/patterns_and_features/all_trial_features.csv')
all_trial_features_valid_lstm = all_trial_features_lstm[(all_trial_features_lstm['t_last_visible']<50) & (all_trial_features_lstm['hitting_arena_edge']==False)].reset_index()
median_values_lstm = all_trial_features_valid_lstm.median(axis=0)

# Appendix of categories

## Cluster around targets

In [ ]:
# category = cluster_around_target_trials
# category = category[category > num_trials]
# category_name = 'cluster_around_target'
# num_trials = 1


# if len(category) > 0:
#   with basic_func.initiate_plot(7, 7, 100):
#     for currentTrial in category[:trial_total_num]:
#         duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
#         plot_trials.PlotTrials(duration, 
#                    *PlotTrials_args,
#                    **classic_plot_kwargs,
#                    currentTrial = currentTrial,
#                    num_trials = num_trials,                   
#                    trail_color_var = "speed",
#                    show_path_when_target_visible = True,
#                    show_connect_path_ff = True,
#                    trial_to_show_cluster_around_target = "current",                 
#                    )

#         temp_animation_plot_kwargs = animation_plot_kwargs.copy()
#         temp_animation_plot_kwargs['trial_to_show_cluster_around_target'] = 'current'
#         data_item.make_animation(currentTrial=currentTrial, num_trials=num_trials, animation_plot_kwargs=temp_animation_plot_kwargs, save_video=False, video_dir=None, file_name=None)
        

#         plt.show()  
#         


## Cluster of fireflies

In [ ]:
# category = cluster_exist_trials
# category = category[category > num_trials]
# category_name = 'cluster_exist_trials'
# num_trials = 1

# if len(category) > 0:
#   with basic_func.initiate_plot(7, 7, 100):
#     for currentTrial in category[:trial_total_num]:
#         duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        
#         plot_trials.PlotTrials(duration, 
                  #  *PlotTrials_args,
                  #  **classic_plot_kwargs,
                  #  currentTrial = currentTrial,
                  #  num_trials = num_trials,                   
#                    trail_color_var = "speed",                 
#                    show_path_when_target_visible = True,
#                    show_connect_path_ff_except_targets = True,
#                    trial_to_show_cluster = "current", 
#                    cluster_dataframe_point = cluster_dataframe_point,
#                    
#                    )

#         plt.show()  
#         


## Waste cluster

In [ ]:
# category = waste_cluster_trials
# category = category[category > num_trials]
# category_name = 'waste_cluster_trials'
# num_trials = 2

# if len(category) > 0:
#   with basic_func.initiate_plot(7, 7, 100):
#     for currentTrial in category[:trial_total_num]:
#         duration = [ff_caught_T_sorted[currentTrial-num_trials], ff_caught_T_sorted[currentTrial]]
        
#         plot_trials.PlotTrials(duration, 
                  #  *PlotTrials_args,
                  #  **classic_plot_kwargs,
                  #  currentTrial = currentTrial,
                  #  num_trials = num_trials,
#                    show_connect_path_ff = True,
#                    trial_to_show_cluster = "previous", 
#                    cluster_dataframe_point = cluster_dataframe_point
#                    
#                    )

#         plt.show()  
#         


# Debug

In [ ]:
!pip install -Uqq ipdb
import ipdb
%pdb on

In [ ]:
%pdb off